In [2]:
import pandas as pd
import numpy as np
import yfinance as yf

In [3]:
#Motley-fool data with transcript for different tickers (company code) and dates
earnings_call_data = pd.read_pickle(r"C:\Users\ruchi\OneDrive\Ruchira-MAIN\Work\Stats_ML\motley-fool-data.pkl\motley-fool-data.pkl")

#adjusting the date format and aligning with market closing time date
earnings_call_data['date'] = earnings_call_data['date'].str.replace(r'\s*ET$', '', regex=True)
earnings_call_data['date'] = earnings_call_data['date'].str.replace('a.m.', 'AM', regex=False).str.replace('p.m.', 'PM', regex=False).str.strip()
earnings_call_data['date'] = pd.to_datetime(earnings_call_data['date'])
earnings_call_data['adjusted_date'] = earnings_call_data['date'].apply(lambda x: x + pd.offsets.BDay(1) if x.hour >= 16 else x)
earnings_call_data['adjusted_date'] = earnings_call_data['adjusted_date'].dt.normalize()
earnings_call_data['adjusted_date']=pd.to_datetime(earnings_call_data['adjusted_date'])


C:\Users\ruchi\AppData\Local\Temp\ipykernel_41440\2414088161.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  earnings_call_data['date'] = pd.to_datetime(earnings_call_data['date'])


In [4]:
earnings_call_data.head()

,date,exchange,q,ticker,transcript,adjusted_date
0,2020-08-27 21:00:00,NASDAQ: BILI,2020-Q2,BILI,"Prepared Remarks:\nOperator\nGood day, and wel...",2020-08-28
1,2020-07-30 16:30:00,NYSE: GFF,2020-Q3,GFF,Prepared Remarks:\nOperator\nThank you for sta...,2020-07-31
2,2019-10-23 17:00:00,NASDAQ: LRCX,2020-Q1,LRCX,Prepared Remarks:\nOperator\nGood day and welc...,2019-10-24
3,2019-11-06 12:00:00,NASDAQ: BBSI,2019-Q3,BBSI,"Prepared Remarks:\nOperator\nGood day, everyon...",2019-11-06
4,2019-08-07 08:30:00,NASDAQ: CSTE,2019-Q2,CSTE,Prepared Remarks:\nOperator\nGreetings and wel...,2019-08-07


In [5]:
#defining start and end date to get the time range for the dataset we have 
start_date=earnings_call_data['adjusted_date'].min().strftime('%Y-%m-%d')
end_date=earnings_call_data['adjusted_date'].max().strftime('%Y-%m-%d')
tickers = earnings_call_data['ticker'].unique().tolist()

In [6]:
#getting the stock prices data from yfinance for the data range above
stock_prices = yf.download(tickers, start=start_date, end=end_date)


$PINC: possibly delisted; no price data found  (1d 2019-04-11 -> 2023-02-23)
[                       0%                       ]  9 of 2876 completed$DRQ: possibly delisted; no timezone found
[                       0%                       ]  10 of 2876 completed$ICPT: possibly delisted; no timezone found
[                       1%                       ]  18 of 2876 completed$CTK: possibly delisted; no timezone found
[                       1%                       ]  19 of 2876 completed$TCS: possibly delisted; no timezone found
[                       1%                       ]  20 of 2876 completed$NTCO: possibly delisted; no timezone found
[                       1%                       ]  21 of 2876 completed$OCSI: possibly delisted; no timezone found
[                       1%                       ]  22 of 2876 completed$ENLC: possibly delisted; no timezone found
[                       1%                       ]  31 of 2876 completed$GLT: possibly delisted; no timezone found


In [7]:
#extracting just the stock prices from the data set above
adjusted_close_prices = stock_prices['Close']
adjusted_close_prices.tail()

Ticker,A,AA,AAIC,AAL,AAN,AAOI,AAON,AAP,AAPL,AAT,...,ZLND.Y,ZM,ZNGA,ZS,ZTS,ZUMZ,ZUO,ZVIA,ZY,ZYXI
Date,,,,,,,,,,,,,,,,,,,,,
2023-02-15,NaN,NaN,NaN,16.799999,NaN,2.83,NaN,142.273819,NaN,22.068245,...,NaN,NaN,NaN,142.770004,169.882980,NaN,NaN,4.50,NaN,12.97
2023-02-16,NaN,NaN,NaN,16.389999,NaN,2.95,NaN,141.239365,NaN,21.918571,...,NaN,NaN,NaN,132.929993,168.824982,NaN,NaN,4.49,NaN,12.89
2023-02-17,NaN,NaN,NaN,16.360001,NaN,2.80,NaN,141.051300,NaN,21.868683,...,NaN,NaN,NaN,132.350006,166.980743,NaN,NaN,4.47,NaN,13.05
2023-02-21,NaN,NaN,NaN,15.690000,NaN,2.72,NaN,135.596954,NaN,21.186842,...,NaN,NaN,NaN,127.839996,164.855011,NaN,NaN,4.24,NaN,12.61
2023-02-22,NaN,NaN,NaN,15.560000,NaN,2.82,NaN,135.060913,NaN,21.403040,...,NaN,NaN,NaN,132.160004,161.545090,NaN,NaN,4.37,NaN,12.54


#### Only SP500

In [8]:
folder =r"C:\Users\ruchi\OneDrive\Ruchira-MAIN\Work\Stats_ML\stock_data"
start_date = earnings_call_data['adjusted_date'].min().strftime('%Y-%m-%d')
end_date = "2022-12-12"

dates = pd.date_range(start_date, end_date, freq="D")


In [9]:
import os
sp500_tickers = []
for file in os.listdir(folder):
    ticker = os.path.splitext(file)[0]
    sp500_tickers.append(ticker)
    required_tickers = [t for t in sp500_tickers if t in tickers]
    

In [ ]:
series_by_ticker = {} # dictionary to hold ticker prices for each ticker

for t in required_tickers:
    path = os.path.join(folder, f"{t}.csv")
    df = pd.read_csv(path, parse_dates = ["Date"], infer_datetime_format=True)
    adj_price = df[["Date", "Adj Close"]]
    s = adj_price.set_index('Date')["Adj Close"].rename(t)
    series_by_ticker[t] = s


C:\Users\ruchi\AppData\Local\Temp\ipykernel_41440\2167643534.py:5: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv(path, parse_dates = ["Date"], infer_datetime_format=True)
C:\Users\ruchi\AppData\Local\Temp\ipykernel_41440\2167643534.py:5: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv(path, parse_dates = ["Date"], infer_datetime_format=True)
C:\Users\ruchi\AppData\Local\Temp\ipykernel_41440\2167643534.py:5: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict

In [ ]:
prices_df = pd.DataFrame(series_by_ticker) #dataframe with all ticker prices


In [ ]:
prices_df.index = pd.to_datetime(prices_df.index).normalize()
start = pd.to_datetime(start_date) - pd.tseries.offsets.BDay(20)
end = pd.to_datetime(end_date)

# slice by datetime range
new_df = prices_df.loc[start:end]
# Calculate 1-day and 5-day returns for each ticker
for ticker in new_df.columns:
    new_df[f'{ticker}_bcallday_r1'] = new_df[ticker].pct_change(1).shift(1)
    new_df[f'{ticker}_r1d'] = new_df[ticker].shift(-1) / new_df[ticker] - 1
    new_df[f'{ticker}_r5d'] = new_df[ticker].shift(-5) / new_df[ticker] - 1

C:\Users\ruchi\AppData\Local\Temp\ipykernel_41440\1535813316.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df[f'{ticker}_callday_r1'] = new_df[ticker].pct_change(1)
C:\Users\ruchi\AppData\Local\Temp\ipykernel_41440\1535813316.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df[f'{ticker}_r1d'] = new_df[ticker].shift(-1) / new_df[ticker] - 1
C:\Users\ruchi\AppData\Local\Temp\ipykernel_41440\1535813316.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [26]:
new_df.tail()

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACGL,ACN,...,ZBRA_r5d,ZBRA_vol20d,ZION_callday_r1,ZION_r1d,ZION_r5d,ZION_vol20d,ZTS_callday_r1,ZTS_r1d,ZTS_r5d,ZTS_vol20d
Date,,,,,,,,,,,,,,,,,,,,,
2022-12-06,151.350006,14.33,145.529999,142.910004,163.720001,169.320007,377.950012,103.860001,60.540001,285.410004,...,NaN,0.036721,-0.026435,-0.001909,NaN,0.030903,-0.014678,-0.018295,NaN,0.024984
2022-12-07,153.729996,13.55,146.649994,140.940002,165.399994,170.289993,377.790009,104.809998,60.150002,286.690002,...,NaN,0.036710,-0.001909,0.007014,NaN,0.030611,-0.018295,0.022829,NaN,0.025256
2022-12-08,156.279999,13.60,146.589996,142.649994,165.990005,170.460007,380.750000,106.919998,60.560001,292.149994,...,NaN,0.035657,0.007014,0.002322,NaN,0.030614,0.022829,-0.001887,NaN,0.024676
2022-12-09,152.949997,13.53,145.899994,142.160004,163.059998,165.330002,380.750000,107.510002,60.040001,288.410004,...,NaN,0.024306,0.002322,0.019162,NaN,0.023124,-0.001887,-0.003455,NaN,0.019696
2022-12-12,155.330002,14.20,145.929993,144.490005,165.320007,167.199997,380.010010,109.489998,60.000000,292.339996,...,NaN,0.023361,0.019162,NaN,NaN,0.022499,-0.003455,NaN,NaN,0.017609


In [23]:
spy_series = yf.download("SPY", start=(start- pd.tseries.offsets.BDay(20)).strftime("%Y-%m-%d") , end=(end + pd.Timedelta(days=1)).strftime("%Y-%m-%d"), progress=False)["Close"]

# normalize index, sort, compute returns
spy_series.index = pd.to_datetime(spy_series.index).normalize()
spy_series = spy_series.sort_index()


In [ ]:
spy_series["m_r1d"] = spy_series["SPY"].shift(-1) / spy_series["SPY"] - 1
spy_series["m_r5d"] = spy_series["SPY"].shift(-5) / spy_series["SPY"] - 1
spy_series["m_bcallday_r1"] = spy_series["SPY"].pct_change(1).shift(1)

In [28]:
df_with_market = pd.merge(new_df, spy_series, left_index = True, right_index = True ,how = "left")

In [29]:
df_with_market

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACGL,ACN,...,ZION_vol20d,ZTS_callday_r1,ZTS_r1d,ZTS_r5d,ZTS_vol20d,SPY,m_r1d,m_r5d,m_vol20d,m_callday_r1
Date,,,,,,,,,,,,,,,,,,,,,
2019-03-14,78.459381,31.704929,146.551392,44.523067,66.664124,74.894989,325.609985,74.421806,32.549999,156.109482,...,NaN,NaN,0.007381,0.035759,NaN,252.856674,0.004940,0.017158,NaN,-0.000640
2019-03-15,78.936333,31.014189,147.350800,45.102238,67.823143,75.526093,333.230011,75.250999,32.869999,157.596481,...,NaN,0.007381,0.008874,0.010525,NaN,254.105896,0.003626,-0.007323,0.005605,0.004940
2019-03-18,78.809799,30.964851,153.755264,45.562660,67.247810,75.723907,333.250000,74.280457,32.840000,157.634354,...,NaN,0.008874,0.009717,0.001636,NaN,255.027206,0.000248,-0.011653,0.005182,0.003626
2019-03-19,79.033638,31.172070,155.515793,45.201588,67.389565,76.364426,339.359985,75.345245,32.360001,157.416519,...,NaN,0.009717,-0.002229,0.001013,NaN,255.090485,-0.003010,-0.004532,0.005183,0.000248
2019-03-20,78.761116,30.550407,156.210449,45.596592,67.064362,74.668900,339.660004,75.326401,31.980000,157.482819,...,NaN,-0.002229,0.011573,0.000203,NaN,254.322708,0.011295,-0.006748,0.005255,-0.003010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-06,151.350006,14.330000,145.529999,142.910004,163.720001,169.320007,377.950012,103.860001,60.540001,285.410004,...,0.030903,-0.014678,-0.018295,NaN,0.024984,376.844635,-0.001701,NaN,0.017385,-0.014415
2022-12-07,153.729996,13.550000,146.649994,140.940002,165.399994,170.289993,377.790009,104.809998,60.150002,286.690002,...,0.030611,-0.018295,0.022829,NaN,0.025256,376.203522,0.007834,NaN,0.017383,-0.001701
2022-12-08,156.279999,13.600000,146.589996,142.649994,165.990005,170.460007,380.750000,106.919998,60.560001,292.149994,...,0.030614,0.022829,-0.001887,NaN,0.024676,379.150696,-0.007470,NaN,0.016620,0.007834


In [ ]:
for t in required_tickers:
    df_with_market[f'{t}_abn_r1d']=df_with_market[f'{t}_r1d']-df_with_market['m_r1d']
    df_with_market[f'{t}_abn_r5d']=df_with_market[f'{t}_r5d']-df_with_market['m_r5d']
    df_with_market[f'{t}_abn_bcallday_r1']=df_with_market[f'{t}_bcallday_r1']-df_with_market['m_bcallday_r1']
    df_with_market[f'{t}_abn_vol20d'] = df_with_market[f'{t}_abn_r1d'].rolling(window=20).std()
    

In [223]:
# convert wide df_with_market columns like "TICK_abn_r1d" -> long table and merge
df_with_market.index = pd.to_datetime(df_with_market.index).normalize()
earnings_call_data['adjusted_date'] = pd.to_datetime(earnings_call_data['adjusted_date']).dt.normalize()

# 1d abnormal returns long
abn1 = df_with_market.filter(regex=r'_abn_r1d$').copy()
abn1.columns = [c.rsplit('_abn_r1d', 1)[0] for c in abn1.columns]   # rename cols -> ticker
abn1_long = abn1.stack().rename('abret_1d').reset_index()
abn1_long.columns = ['adjusted_date', 'ticker', 'abret_1d']

# 5d abnormal returns long
abn5 = df_with_market.filter(regex=r'_abn_r5d$').copy()
abn5.columns = [c.rsplit('_abn_r5d', 1)[0] for c in abn5.columns]
abn5_long = abn5.stack().rename('abret_5d').reset_index()
abn5_long.columns = ['adjusted_date', 'ticker', 'abret_5d']

# merge both into earnings_call_data
earnings_call_data = earnings_call_data.merge(
    abn1_long, on=['adjusted_date', 'ticker'], how='left'
).merge(
    abn5_long, on=['adjusted_date', 'ticker'], how='left'
)

In [224]:
earnings_call_data

,date,exchange,q,ticker,transcript,adjusted_date,abret_1d,abret_5d
0,2020-08-27 21:00:00,NASDAQ: BILI,2020-Q2,BILI,"Prepared Remarks:\nOperator\nGood day, and wel...",2020-08-28,NaN,NaN
1,2020-07-30 16:30:00,NYSE: GFF,2020-Q3,GFF,Prepared Remarks:\nOperator\nThank you for sta...,2020-07-31,NaN,NaN
2,2019-10-23 17:00:00,NASDAQ: LRCX,2020-Q1,LRCX,Prepared Remarks:\nOperator\nGood day and welc...,2019-10-24,0.137401,0.121305
3,2019-11-06 12:00:00,NASDAQ: BBSI,2019-Q3,BBSI,"Prepared Remarks:\nOperator\nGood day, everyon...",2019-11-06,NaN,NaN
4,2019-08-07 08:30:00,NASDAQ: CSTE,2019-Q2,CSTE,Prepared Remarks:\nOperator\nGreetings and wel...,2019-08-07,NaN,NaN
...,...,...,...,...,...,...,...,...
18750,2021-11-09 13:00:00,NYSE: SWX,2021-Q3,SWX,Prepared Remarks:\nOperator\nLadies and gentle...,2021-11-09,NaN,NaN
18751,2021-11-18 12:00:00,NYSE: PNNT,2021-Q4,PNNT,"Prepared Remarks:\nOperator\nGood morning, and...",2021-11-18,NaN,NaN
18752,2022-02-08 11:00:00,NYSE: TDG,2022-Q1,TDG,Prepared Remarks:\nOperator\nThank you for sta...,2022-02-08,0.011638,0.038610
18753,2022-02-28 16:30:00,NASDAQ: DVAX,2021-Q4,DVAX,"Prepared Remarks:\nOperator\nGood day, ladies ...",2022-03-01,NaN,NaN


In [225]:
clean_df=earnings_call_data.dropna()
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3546 entries, 2 to 18752
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           3546 non-null   datetime64[ns]
 1   exchange       3546 non-null   object        
 2   q              3546 non-null   object        
 3   ticker         3546 non-null   object        
 4   transcript     3546 non-null   object        
 5   adjusted_date  3546 non-null   datetime64[ns]
 6   abret_1d       3546 non-null   float64       
 7   abret_5d       3546 non-null   float64       
dtypes: datetime64[ns](2), float64(2), object(4)
memory usage: 249.3+ KB


In [239]:
clean_df['r1d_direction'] = (clean_df['abret_1d'] > 0).astype(int)
clean_df['r5d_direction'] = (clean_df['abret_5d'] > 0).astype(int)

C:\Users\ruchi\AppData\Local\Temp\ipykernel_33768\443728299.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df['r1d_direction'] = (clean_df['abret_1d'] > 0).astype(int)
C:\Users\ruchi\AppData\Local\Temp\ipykernel_33768\443728299.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df['r5d_direction'] = (clean_df['abret_5d'] > 0).astype(int)


In [240]:
final_df = clean_df[["adjusted_date" , "ticker", "transcript", "abret_1d" , "abret_5d","r1d_direction","r5d_direction"]]

In [250]:
final_df.to_pickle(r"C:\Users\ruchi\OneDrive\Ruchira-MAIN\Work\Stats_ML\model_input_data.pkl")